INSTALLS AND IMPORTS

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks
!pip install -U langchain
!pip install langchain-groq
!pip install -qU langchain-huggingface
!pip install groq


In [ ]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
import getpass

# **API KEYS**

Login and create the GROQ_API_KEY in https://console.groq.com/keys

In [ ]:
GROQ_API_KEY = "YOUR GROQ KEY"

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("INSERT YOUR GROQ API KEY:")

PINECONE_KEY = "YOR PINECONE KEY"
pc = Pinecone(api_key=PINECONE_KEY)



# **CREATE YOUR INDEX AND VECTOR**

In [ ]:
index = pc.Index("YOUR INDEX")
index_name = "YOUR INDEX NAME"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# **SEARCH SIMILARITY IN VECTORS**

In [ ]:
def search_in_db(query):
    results = vector_store.similarity_search(
        query,
        k=4
    )
    response = []
    for res in results:
        response.append(f"* {res.page_content} [{res.metadata}]")

    return response

# **DESCRIBE THE PROMPT FROM YOUR TEMPLATE**

Insert the context in the prompt_template.

Example:

prompt_template= """You are an artificial intelligence model that assists customers.
Follow these steps:
1) Explore the information about the company found in the PDF.
2) Provide a brief and concise answer.
3) If the question is not covered in the context, respond with the following phrase: 'I don’t have context to answer your question.
Heres the contex :
{context} """

In [ ]:
prompt_template = """
YOU'RE AN EXPERT IN ...
Follow the next steps:
1 ) INSERT AN STEP.
2 ) INSERT AN STEP.
3) INSERT AN STEP.
Heres the contex :
{context}
"""

# **CREATE THE LARGE LANGUAGE MODEL**

In [ ]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key=GROQ_API_KEY
)

In [ ]:
query = input("Ask something")
context = search_in_db(query)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_template,
        ),
        ("human", "{question}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "context": context,
        "question": query,
    }
)